In [1]:
import requests

print(requests.__version__)  # This prints the version of the requests library
print(dir(requests.exceptions))  # This prints the directory of the requests.exceptions module, verifying its existence



2.31.0
['BaseHTTPError', 'ChunkedEncodingError', 'CompatJSONDecodeError', 'ConnectTimeout', 'ConnectionError', 'ContentDecodingError', 'FileModeWarning', 'HTTPError', 'InvalidHeader', 'InvalidJSONError', 'InvalidProxyURL', 'InvalidSchema', 'InvalidURL', 'JSONDecodeError', 'MissingSchema', 'ProxyError', 'ReadTimeout', 'RequestException', 'RequestsDependencyWarning', 'RequestsWarning', 'RetryError', 'SSLError', 'StreamConsumedError', 'Timeout', 'TooManyRedirects', 'URLRequired', 'UnrewindableBodyError', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


In [2]:
import ee

# Initialize the Earth Engine library.
ee.Initialize()


In [3]:
# Fetch a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Print the image metadata.
print(image.getInfo())


{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7661, 7801], 'crs': 'EPSG:32610', 'crs_transform': [30, 0, 460785, 0, -30, 4264215]}, {'

In [4]:
#import folium` is importing the Folium library in Python. Folium is a Python library used for visualizing geospatial data interactively on an interactive map.
import folium
import ee

ee.Initialize()

# Define a function to add Earth Engine layers to a Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine layer method to folium.Map.
folium.Map.add_ee_layer = add_ee_layer

# Example usage
# Define a location: Johannesburg, South Africa
location = [-26.2041, 28.0473]

# Create a folium map object.
my_map = folium.Map(location=location, zoom_start=10)

# Fetch a Landsat 8 image.
image = ee.Image('LANDSAT/LC08/C01/T1/LC08_044034_20140318')

# Set visualization parameters.
vis_params = {
    'bands': ['B4', 'B3', 'B2'],  # RGB bands
    'min': 0,
    'max': 30000,
    'gamma': 1.4
}

# Add the Landsat 8 image to the map object.
my_map.add_ee_layer(image, vis_params, 'Landsat 8')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
my_map





In [7]:
import ee
import folium

ee.Initialize()

# Define the area of interest: Johannesburg, South Africa.
aoi = ee.Geometry.Point([28.0473, -26.2041]).buffer(10000)  # Buffer by 10 km

# Load datasets and mean over the specified period for simplicity.
lst = ee.ImageCollection('MODIS/006/MOD11A1').select('LST_Day_1km').mean().clip(aoi)  # Mean LST
ndvi = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI').mean().clip(aoi)  # Mean NDVI
# Example for South Africa, year 2020
#population = ee.Image("WorldPop/GP/100m/ZAF_2020").clip(aoi)

# Normalize each dataset. The normalization is performed by subtracting the minimum
# of the dataset and dividing by the range of the dataset.
# These operations are kept server-side.
def normalize(image, region, scale=1000):
    minVal = ee.Number(image.reduceRegion(ee.Reducer.min(), region, scale).values().get(0))
    maxVal = ee.Number(image.reduceRegion(ee.Reducer.max(), region, scale).values().get(0))
    return image.subtract(minVal).divide(maxVal.subtract(minVal))

lst_norm = normalize(lst, aoi)
ndvi_norm = normalize(ndvi, aoi)
#pop_norm = normalize(population, aoi)

# Combine the normalized factors to create the HVI. Here, NDVI is subtracted because higher NDVI values
# (more vegetation) typically mean lower heat vulnerability, contrary to LST and population density.
# This is a simple arithmetic mean of the factors to create the index.
hvi = lst_norm.add(ndvi_norm.multiply(-1)).divide(2)

# Visualization parameters for HVI.
vis_params = {
    'min': 0,
    'max': 1,
    'palette': ['blue', 'white', 'red']
}

# Function to add Earth Engine layers to Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

# Create a Folium map centered on the area of interest.
map = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)
map.add_ee_layer(hvi, vis_params, 'Heat Vulnerability Index')

# Add a layer control panel to toggle layers on and off.
map.add_child(folium.LayerControl())

# Display the map.
map


In [8]:
import folium
import ee

ee.Initialize()

# Define the area of interest: Johannesburg, South Africa.
aoi = ee.Geometry.Point([28.0473, -26.2041]).buffer(10000)  # Buffer by 10 km

# Function to add Earth Engine layers to Folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine layer method to folium.Map.
folium.Map.add_ee_layer = add_ee_layer

# Example known-good layer (MODIS NDVI)
modis_ndvi = ee.ImageCollection('MODIS/006/MOD13Q1').select('NDVI').mean().clip(aoi)
ndvi_vis_params = {
    'min': 0,
    'max': 9000,
    'palette': ['00FF00', 'FF0000']
}

# Create a Folium map centered on the area of interest.
simple_map = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)

# Add the MODIS NDVI layer to the map using the corrected variable name for visualization parameters.
simple_map.add_ee_layer(modis_ndvi, ndvi_vis_params, 'MODIS NDVI')

# Display the simplified map.
simple_map.add_child(folium.LayerControl())
simple_map



In [9]:
# Attempt to get Johannesburg boundaries from GADM data (administrative boundaries)
# Note: Adjust "ZAF" and administrative level as necessary to better fit Johannesburg boundaries.
johannesburg_boundaries = ee.FeatureCollection('FAO/GAUL/2015/level2').filter(ee.Filter.eq('ADM2_NAME', 'Johannesburg'))

# To visualize and confirm boundaries, you might want to display this layer first.


In [10]:
# Assuming `hvi` is your final Heat Vulnerability Index image:
hvi_johannesburg = hvi.clip(johannesburg_boundaries)


In [11]:
# Assuming `lst`, `ndvi`, and possibly `population` are defined earlier
try:
    # Check if lst_norm evaluates to an image
    print(lst_norm.getInfo())
    
    # Check if ndvi_norm evaluates to an image
    print(ndvi_norm.getInfo())
    
    # If using population data
    # print(pop_norm.getInfo())
except Exception as e:
    print(e)


{'type': 'Image', 'bands': [{'id': 'LST_Day_1km', 'data_type': {'type': 'PixelType', 'precision': 'double', 'min': -83.22128864612405, 'max': 283.3800330045181}, 'dimensions': [2, 2], 'origin': [27, -27], 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:footprint': {'type': 'Polygon', 'coordinates': [[[28.0473, -26.114111340485184], [28.019306616602588, -26.117690180576375], [27.99353669130437, -26.128142452689524], [27.972038190924838, -26.144637843859336], [27.956521606502278, -26.165865545252068], [27.948224048435236, -26.19013790088324], [27.94781015862496, -26.21552409985472], [27.955317913685608, -26.240003471206084], [27.97015403235698, -26.261626304440053], [27.991139842273643, -26.27866940413439], [28.01660438259738, -26.289773886250117], [28.04451756112857, -26.29405405080564], [28.072652723210197, -26.291168442906578], [28.098765387075858, -26.281347269201113], [28.120773447805355, -26.265373912599795], [28.136924019334938, -26.24452206754058

In [12]:
# Assuming `johannesburg_boundaries` is a FeatureCollection outlining the city or wards
hvi_johannesburg = hvi.clip(johannesburg_boundaries)

# Visualization parameters, adjusted for NDVI as an example
ndvi_vis_params = {
    'min': 0.2,  # Adjust based on your NDVI data to enhance visibility
    'max': 0.8,
    'palette': ['blue', 'white', 'green']
}

# Initialize a folium map centered on Johannesburg
map_johannesburg = folium.Map(location=[-26.2041, 28.0473], zoom_start=11)

# Add the NDVI layer with enhanced visibility
map_johannesburg.add_ee_layer(modis_ndvi.clip(johannesburg_boundaries), ndvi_vis_params, 'Enhanced NDVI')

# Add the HVI layer
map_johannesburg.add_ee_layer(hvi_johannesburg, {
    'min': 0,
    'max': 1,
    'palette': ['yellow', 'red']
}, 'Heat Vulnerability Index')

# Optionally, add Johannesburg boundaries for reference
# Ensure 'johannesburg_boundaries' is styled appropriately for visibility
map_johannesburg.add_ee_layer(johannesburg_boundaries.style(**{
    'color': 'black',
    'fillColor': '00000000'  # Transparent fill
}), {}, 'Johannesburg Boundaries')

# Add a layer control panel
map_johannesburg.add_child(folium.LayerControl())

# Display the map
map_johannesburg



In [13]:
import folium
import ee

ee.Initialize()

# Define your area of interest or load Johannesburg boundaries
# For demonstration, let's assume johannesburg_boundaries is already defined

# Create an empty image onto which the boundaries will be drawn.
empty = ee.Image().byte()

# Draw the boundaries as lines on the empty image.
boundary_image = empty.paint(**{
    'featureCollection': johannesburg_boundaries,
    'color': 1,  # This is a nominal value for the paint operation.
    'width': 3  # Width of the outline.
})

# Set visualization parameters for the boundaries.
vis_params = {
    'palette': ['red'],
    'min': 0,
    'max': 3,
}

# Initialize a Folium map centered on Johannesburg.
map_johannesburg = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)

# Define the function to add Earth Engine layers to Folium map if not already defined.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data © Google Earth Engine',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add this method to folium.Map objects
folium.Map.add_ee_layer = add_ee_layer

# Now, use this function to add the boundary image as a layer.
map_johannesburg.add_ee_layer(boundary_image, vis_params, 'Johannesburg Boundaries')

# Add a layer control panel.
map_johannesburg.add_child(folium.LayerControl())

# Display the map.
map_johannesburg



In [14]:
import fiona

# Path to the file geodatabase, it should be the directory containing the .gdb files
gdb_path = 'ZAF_AdminBoundaries_candidate.gdb'

# List all layers in the geodatabase
layers = fiona.listlayers(gdb_path)

# Print all layer names
for layer in layers:
    print(layer)



zaf_admbndt_adminUnitLookup
zaf_admbnda_adm4_sadb_ocha_20201109
zaf_admbndl_admALL_sadb_ocha_itos_20201109
zaf_admbnda_adm0_sadb_ocha_20201109
zaf_admbnda_adm1_sadb_ocha_20201109
zaf_admbnda_adm2_sadb_ocha_20201109
zaf_admbnda_adm3_sadb_ocha_20201109
zaf_admbndp_admALL_sadb_ocha_itos_20201109


In [16]:
import geopandas as gpd

# Replace with the path to your geodatabase and the layer name
gdb_path = 'ZAF_AdminBoundaries_candidate.gdb'
layer_name = 'zaf_admbnda_adm4_sadb_ocha_20201109'

# Load the layer into a GeoDataFrame
adm4_gdf = gpd.read_file(gdb_path, layer=layer_name)

# Now adm4_gdf is defined, you can print the columns and the first few rows
print(adm4_gdf.columns)
print(adm4_gdf.head())



Index(['admin4Name_en', 'admin4Pcode', 'admin4RefName', 'admin4AltName1_en',
       'admin4AltName2_en', 'admin3Name_en', 'admin3Pcode', 'admin2Name_en',
       'admin2Pcode', 'admin1Name_en', 'admin1Pcode', 'admin0Name_en',
       'admin0Pcode', 'date', 'validOn', 'validTo', 'ADM1_ID', 'ADM2_ID',
       'ADM3_ID', 'Shape_Length', 'Shape_Area', 'geometry'],
      dtype='object')
  admin4Name_en admin4Pcode admin4RefName admin4AltName1_en admin4AltName2_en  \
0           001   ZA9344001          None              None              None   
1           002   ZA9344002          None              None              None   
2           003   ZA9344003          None              None              None   
3           004   ZA9344004          None              None              None   
4           005   ZA9344005          None              None              None   

  admin3Name_en admin3Pcode admin2Name_en admin2Pcode admin1Name_en  ...  \
0       Makhado      ZA9344        Vhembe       ZA934  

In [17]:
# Filter the GeoDataFrame for Johannesburg wards using the 'admin2Name_en' column
johannesburg_wards = adm4_gdf[adm4_gdf['admin2Name_en'].str.contains('Johannesburg', na=False)]

# Display the first few rows to inspect
print(johannesburg_wards.head())


     admin4Name_en admin4Pcode admin4RefName admin4AltName1_en  \
1784           118   ZA7980118          None              None   
1903           044   ZA7980044          None              None   
1904           045   ZA7980045          None              None   
1905           046   ZA7980046          None              None   
1906           047   ZA7980047          None              None   

     admin4AltName2_en         admin3Name_en admin3Pcode  \
1784              None  City of Johannesburg      ZA7980   
1903              None  City of Johannesburg      ZA7980   
1904              None  City of Johannesburg      ZA7980   
1905              None  City of Johannesburg      ZA7980   
1906              None  City of Johannesburg      ZA7980   

             admin2Name_en admin2Pcode admin1Name_en  ... admin0Pcode  \
1784  City of Johannesburg       ZA798       Gauteng  ...          ZA   
1903  City of Johannesburg       ZA798       Gauteng  ...          ZA   
1904  City of Johannesb

In [18]:
johannesburg_wards.to_file('johannesburg_wards.geojson', driver='GeoJSON')

In [31]:
import folium
import geopandas as gpd

# Assuming 'johannesburg_wards' is loaded with geopandas

# Create a base map
m = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)

# Add only the geometry to the map, ignoring other attributes
for _, row in johannesburg_wards.iterrows():
    # Simplifying the geometry to ensure smooth rendering
    simpl = row['geometry'].simplify(tolerance=0.001)
    geoj = folium.GeoJson(data=simpl,
                          style_function=lambda x: {'fillColor': 'green', 'color': 'green'})
    geoj.add_to(m)

# Show map
m








In [28]:
import geopandas as gpd

# Assuming johannesburg_wards is your GeoDataFrame
print(johannesburg_wards.columns)



Index(['admin4Name_en', 'admin4Pcode', 'admin4RefName', 'admin4AltName1_en',
       'admin4AltName2_en', 'admin3Name_en', 'admin3Pcode', 'admin2Name_en',
       'admin2Pcode', 'admin1Name_en', 'admin1Pcode', 'admin0Name_en',
       'admin0Pcode', 'date', 'validOn', 'validTo', 'ADM1_ID', 'ADM2_ID',
       'ADM3_ID', 'Shape_Length', 'Shape_Area', 'geometry'],
      dtype='object')


In [30]:
import folium
import geopandas as gpd

# Assuming 'johannesburg_wards' is your GeoDataFrame
# and it's already loaded

# Create a base map
m = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)

# Add the geometry and a popup with the ward name to the map
for _, row in johannesburg_wards.iterrows():
    # Simplifying the geometry for smoother rendering
    simpl = row['geometry'].simplify(tolerance=0.001)
    geoj = folium.GeoJson(data=simpl,
                          style_function=lambda x: {'fillColor': 'green', 'color': 'green'})
    
    # Use 'admin4Name_en' as the popup content
    popup = folium.Popup(row['admin4Name_en'], parse_html=True)
    popup.add_to(geoj)
    
    geoj.add_to(m)

# Show map
m



In [33]:
import geopandas as gpd

# Replace 'your_file_path_here' with the actual path to the .gdb file and
# 'layer_name_here' with the name of the layer you're interested in
file_path = 'ZAF_AdminBoundaries_candidate.gdb'
layer_name = 'zaf_admbnda_adm3_sadb_ocha_20201109'


#layer_name = 'zaf_admbnda_adm4_sadb_ocha_20201109'
#zaf_admbndt_adminUnitLookup
#zaf_admbnda_adm4_sadb_ocha_20201109
#zaf_admbndl_admALL_sadb_ocha_itos_20201109
#zaf_admbnda_adm0_sadb_ocha_20201109
#zaf_admbnda_adm1_sadb_ocha_20201109
#zaf_admbnda_adm2_sadb_ocha_20201109
#zaf_admbnda_adm3_sadb_ocha_20201109
#zaf_admbndp_admALL_sadb_ocha_itos_20201109

# Load the layer into a GeoDataFrame
gdf = gpd.read_file(file_path, layer=layer_name)

# Print the first few rows to see what the data looks like
print(gdf.head())

# Print the column names to understand the attributes each feature has
print(gdf.columns)


   admin3Name_en admin3Pcode admin3RefName admin3AltName1_en  \
0      Abaqulusi      ZA5263          None              None   
1    Alfred Duma      ZA5238          None              None   
2      Amahlathi      ZA2124          None              None   
3  Ba-Phalaborwa      ZA9334          None              None   
4  Beaufort West      ZA1053          None              None   

  admin3AltName2_en  admin2Name_en admin2Pcode  admin1Name_en admin1Pcode  \
0              None       Zululand       ZA526  KwaZulu-Natal         ZA5   
1              None       Uthukela       ZA523  KwaZulu-Natal         ZA5   
2              None       Amathole       ZA212   Eastern Cape         ZA2   
3              None         Mopani       ZA933        Limpopo         ZA9   
4              None  Central Karoo       ZA105   Western Cape         ZA1   

  admin0Name_en admin0Pcode                      date  \
0  South Africa          ZA 2018-12-27 00:00:00+00:00   
1  South Africa          ZA 2018-12-27

In [34]:
import geopandas as gpd

# Load the layer into a GeoDataFrame with a unique variable name
gdb_path_jhb = 'ZAF_AdminBoundaries_candidate.gdb'
layer_name_jhb_municipalities = 'zaf_admbnda_adm3_sadb_ocha_20201109'
gdf_jhb_municipalities = gpd.read_file(gdb_path_jhb, layer=layer_name_jhb_municipalities)

# Filter the GeoDataFrame for Johannesburg municipalities with a unique variable name
# Adjust the filtering condition as needed based on accurate identifiers for Johannesburg municipalities
jhb_municipalities_filtered = gdf_jhb_municipalities[gdf_jhb_municipalities['admin3Name_en'].str.contains('Johannesburg', na=False)]

# Save the filtered GeoDataFrame to a GeoJSON file with a unique file name
jhb_municipalities_geojson_path = 'johannesburg_municipalities_unique.geojson'
jhb_municipalities_filtered.to_file(jhb_municipalities_geojson_path, driver='GeoJSON')


In [36]:
import geopandas as gpd
import folium
import json

# Load the GeoJSON file
jhb_municipalities_path = 'johannesburg_municipalities_unique.geojson'
jhb_municipalities_viz = gpd.read_file(jhb_municipalities_path)

# Convert all datetime columns to string format explicitly
for col in jhb_municipalities_viz.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]']).columns:
    jhb_municipalities_viz[col] = jhb_municipalities_viz[col].astype(str)

# Create a simple visualization map
jhb_viz_map = folium.Map(location=[-26.2041, 28.0473], zoom_start=10)

# Add GeoDataFrame to the map as a GeoJson layer
folium.GeoJson(data=jhb_municipalities_viz,
               style_function=lambda x: {'fillColor': 'blue', 'color': 'blue', 'weight': 2},
               name="Johannesburg Municipalities").add_to(jhb_viz_map)

# Optionally, add a layer control widget
folium.LayerControl().add_to(jhb_viz_map)

# Display the map
jhb_viz_map


